In [1]:
import qiskit
from qiskit import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
registers = QuantumRegister(25)
cregisters = ClassicalRegister(25)

circuit = QuantumCircuit(registers, cregisters)

In [3]:
row_oracle = QuantumCircuit(4, name = 'row_oracle')

row_oracle.h(0)
row_oracle.h(3)
row_oracle.x(0)
row_oracle.x(3)
row_oracle.ccx(0,3,1)
row_oracle.x(0)
row_oracle.x(3)
row_oracle.ccx(0,3,2)
row_oracle.cx(2,0)
row_oracle.cx(2,3)

row_oracle.to_gate()
row_oracle.draw()

┌───┐┌───┐     ┌───┐     ┌───┐     
q_0: ┤ H ├┤ X ├──■──┤ X ├──■──┤ X ├─────
     └───┘└───┘┌─┴─┐└───┘  │  └─┬─┘     
q_1: ──────────┤ X ├───────┼────┼───────
               └─┬─┘     ┌─┴─┐  │       
q_2: ────────────┼───────┤ X ├──■────■──
     ┌───┐┌───┐  │  ┌───┐└─┬─┘     ┌─┴─┐
q_3: ┤ H ├┤ X ├──■──┤ X ├──■───────┤ X ├
     └───┘└───┘     └───┘          └───┘

In [4]:
circuit.append(row_oracle, [0,1,2,3])
circuit.append(row_oracle, [4,5,6,7])
circuit.append(row_oracle, [8,9,10,11])
circuit.append(row_oracle, [12,13,14,15])

In [5]:
circuit.x(registers[19])
circuit.x(registers[20])
circuit.x(registers[21])
circuit.x(registers[22])
circuit.x(registers[23])
circuit.x(registers[24])

#AFTER DRAWING, WE SEE A REPEATED PATTERN ON 4 QUBITS, THIS SETS OUR STATE TO A SINGLE 1 per 4 QUBITS
circuit.draw()

┌─────────────┐
 q0_0: ┤0            ├
       │             │
 q0_1: ┤1            ├
       │  row_oracle │
 q0_2: ┤2            ├
       │             │
 q0_3: ┤3            ├
       ├─────────────┤
 q0_4: ┤0            ├
       │             │
 q0_5: ┤1            ├
       │  row_oracle │
 q0_6: ┤2            ├
       │             │
 q0_7: ┤3            ├
       ├─────────────┤
 q0_8: ┤0            ├
       │             │
 q0_9: ┤1            ├
       │  row_oracle │
q0_10: ┤2            ├
       │             │
q0_11: ┤3            ├
       ├─────────────┤
q0_12: ┤0            ├
       │             │
q0_13: ┤1            ├
       │  row_oracle │
q0_14: ┤2            ├
       │             │
q0_15: ┤3            ├
       └─────────────┘
q0_16: ───────────────
                      
q0_17: ───────────────
                      
q0_18: ───────────────
            ┌───┐     
q0_19: ─────┤ X ├─────
            ├───┤     
q0_20: ─────┤ X ├─────
            ├───┤     
q0_21: ─────┤ X ├─────
            ├───┤     
q0_22: ─────┤ X ├─────
            ├───┤     
q0_23: ─────┤ X ├─────
            ├───┤     
q0_24: ─────┤ X ├─────
            └───┘     
c0: 25/═══════════════

In [6]:
# TIME FOR COLUMN CHECK
circuit.h(16)
circuit.h(17)
circuit.h(18)

pi = 3.1415926536

circuit.cp(pi, 16, 0)
circuit.cp(pi, 16, 4)
circuit.cp(pi, 16, 8)
circuit.cp(pi, 16, 12)
circuit.cp(pi, 17, 1)
circuit.cp(pi, 17, 5)
circuit.cp(pi, 17, 9)
circuit.cp(pi, 17, 13)
circuit.cp(pi, 18, 2)
circuit.cp(pi, 18, 6)
circuit.cp(pi, 18, 10)
circuit.cp(pi, 18, 14)

circuit.h(16)
circuit.h(17)
circuit.h(18)

circuit.draw()

┌─────────────┐                                                        »
 q0_0: ┤0            ├─■──────────────────────────────────────────────────────»
       │             │ │                                                      »
 q0_1: ┤1            ├─┼──────■───────────────────────────────────────────────»
       │  row_oracle │ │      │                                               »
 q0_2: ┤2            ├─┼──────┼──────■────────────────────────────────────────»
       │             │ │      │      │                                        »
 q0_3: ┤3            ├─┼──────┼──────┼────────────────────────────────────────»
       ├─────────────┤ │      │      │                                        »
 q0_4: ┤0            ├─┼──────┼──────┼──────■─────────────────────────────────»
       │             │ │      │      │      │                                 »
 q0_5: ┤1            ├─┼──────┼──────┼──────┼──────■──────────────────────────»
       │  row_oracle │ │      │      │      │      │                          »
 q0_6: ┤2            ├─┼──────┼──────┼──────┼──────┼──────■───────────────────»
       │             │ │      │      │      │      │      │                   »
 q0_7: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼───────────────────»
       ├─────────────┤ │      │      │      │      │      │                   »
 q0_8: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────■────────────»
       │             │ │      │      │      │      │      │      │            »
 q0_9: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────■─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_10: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_11: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       ├─────────────┤ │      │      │      │      │      │      │      │     »
q0_12: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_13: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_14: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_15: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       └────┬───┬────┘ │P(π)  │      │      │P(π)  │      │      │P(π)  │     »
q0_16: ─────┤ H ├──────■──────┼──────┼──────■──────┼──────┼──────■──────┼─────»
            ├───┤             │P(π)  │             │P(π)  │             │P(π) »
q0_17: ─────┤ H ├─────────────■──────┼─────────────■──────┼─────────────■─────»
            ├───┤                    │P(π)                │P(π)               »
q0_18: ─────┤ H ├────────────────────■────────────────────■───────────────────»
            ├───┤                                                             »
q0_19: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_20: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_21: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_22: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_23: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_24: ─────┤ X ├─────────────────────────────────────────────────────────────»
       

In [7]:
#ADD DIAGONAL CHECKS

circuit.ccx(registers[0], registers[5], registers[19])
circuit.ccx(registers[1], registers[4], registers[19])
circuit.ccx(registers[1], registers[6], registers[19])
circuit.ccx(registers[2], registers[5], registers[19])
circuit.ccx(registers[2], registers[7], registers[19])
circuit.ccx(registers[3], registers[6], registers[19])

circuit.ccx(registers[0], registers[10], registers[20])
circuit.ccx(registers[1], registers[11], registers[20])
circuit.ccx(registers[2], registers[8], registers[20])
circuit.ccx(registers[3], registers[9], registers[20])


circuit.ccx(registers[0], registers[15], registers[21])
circuit.ccx(registers[3], registers[12], registers[21])

circuit.ccx(registers[4], registers[9], registers[22])
circuit.ccx(registers[5], registers[8], registers[22])
circuit.ccx(registers[5], registers[10], registers[22])
circuit.ccx(registers[6], registers[9], registers[22])
circuit.ccx(registers[6], registers[11], registers[22])
circuit.ccx(registers[7], registers[10], registers[22])

circuit.ccx(registers[4], registers[14], registers[23])
circuit.ccx(registers[5], registers[15], registers[23])
circuit.ccx(registers[6], registers[12], registers[23])
circuit.ccx(registers[7], registers[13], registers[23])

circuit.ccx(registers[8], registers[13], registers[24])
circuit.ccx(registers[9], registers[12], registers[24])
circuit.ccx(registers[9], registers[14], registers[24])
circuit.ccx(registers[10], registers[13], registers[24])
circuit.ccx(registers[10], registers[15], registers[24])
circuit.ccx(registers[11], registers[14], registers[24])

circuit.measure(registers, cregisters)
circuit.draw()


┌─────────────┐                                                        »
 q0_0: ┤0            ├─■──────────────────────────────────────────────────────»
       │             │ │                                                      »
 q0_1: ┤1            ├─┼──────■───────────────────────────────────────────────»
       │  row_oracle │ │      │                                               »
 q0_2: ┤2            ├─┼──────┼──────■────────────────────────────────────────»
       │             │ │      │      │                                        »
 q0_3: ┤3            ├─┼──────┼──────┼────────────────────────────────────────»
       ├─────────────┤ │      │      │                                        »
 q0_4: ┤0            ├─┼──────┼──────┼──────■─────────────────────────────────»
       │             │ │      │      │      │                                 »
 q0_5: ┤1            ├─┼──────┼──────┼──────┼──────■──────────────────────────»
       │  row_oracle │ │      │      │      │      │                          »
 q0_6: ┤2            ├─┼──────┼──────┼──────┼──────┼──────■───────────────────»
       │             │ │      │      │      │      │      │                   »
 q0_7: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼───────────────────»
       ├─────────────┤ │      │      │      │      │      │                   »
 q0_8: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────■────────────»
       │             │ │      │      │      │      │      │      │            »
 q0_9: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────■─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_10: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_11: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       ├─────────────┤ │      │      │      │      │      │      │      │     »
q0_12: ┤0            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_13: ┤1            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │  row_oracle │ │      │      │      │      │      │      │      │     »
q0_14: ┤2            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       │             │ │      │      │      │      │      │      │      │     »
q0_15: ┤3            ├─┼──────┼──────┼──────┼──────┼──────┼──────┼──────┼─────»
       └────┬───┬────┘ │P(π)  │      │      │P(π)  │      │      │P(π)  │     »
q0_16: ─────┤ H ├──────■──────┼──────┼──────■──────┼──────┼──────■──────┼─────»
            ├───┤             │P(π)  │             │P(π)  │             │P(π) »
q0_17: ─────┤ H ├─────────────■──────┼─────────────■──────┼─────────────■─────»
            ├───┤                    │P(π)                │P(π)               »
q0_18: ─────┤ H ├────────────────────■────────────────────■───────────────────»
            ├───┤                                                             »
q0_19: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_20: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_21: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_22: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_23: ─────┤ X ├─────────────────────────────────────────────────────────────»
            ├───┤                                                             »
q0_24: ─────┤ X ├─────────────────────────────────────────────────────────────»
       

In [ ]:
# from qiskit.providers.basic_provider import BasicSimulator
# from qiskit_aer import AerSimulator
# from qiskit.quantum_info import Statevector

# simulator_aer = AerSimulator()

# qc_aer = transpile(circuit, backend = simulator_aer)
# job = simulator_aer.run(qc_aer)
# result = job.result()
# counts = result.get_statevector()
# print(counts)
